In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()
data1=pd.read_csv(io.BytesIO(uploaded['Train_data.csv']))
test1=pd.read_csv(io.BytesIO(uploaded['Test_data.csv']))

Saving Test_data.csv to Test_data.csv
Saving Train_data.csv to Train_data.csv


In [ ]:
data=data1.copy()
test=test1.copy()
data.shape, test.shape

((50667, 43), (16890, 42))

In [ ]:
data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,b,b,b,b,b,b,o,o,x,b,b,b,x,x,o,x,b,b,b,b,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,loss
1,b,b,b,b,b,b,o,b,b,b,b,b,x,o,o,b,b,b,x,b,b,b,b,b,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,loss
2,x,x,o,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,o,x,x,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,win
3,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,b,b,b,b,b,o,x,o,x,b,b,win
4,x,x,o,o,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,x,x,b,b,b,b,b,b,b,b,b,win


In [ ]:
data = data.replace(['b'],0)
data = data.replace(['x'],1)
data = data.replace(['o'],-1)

test = test.replace(['b'],0)
test = test.replace(['x'],1)
test = test.replace(['o'],-1)

In [ ]:
data['class']=data['class'].replace(['loss','win','draw'],[0,1,2])

In [ ]:
data['class'].value_counts()

1    33354
0    12476
2     4837
Name: class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train,localtest, = train_test_split(
    data,
    test_size=0.3,
    random_state=0,
    stratify=data['class']  
    )
train.shape, localtest.shape

((35466, 43), (15201, 43))

In [ ]:
X_train=train.drop(columns=['class'],axis=1)
y_train=train['class']
X_localtest=localtest.drop(columns=['class'],axis=1)
y_localtest=localtest['class']

In [ ]:
from sklearn.utils import class_weight
a=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
weights={0:a[0],1:a[1],2:a[2]}
weights

{0: 1.3537157906790336, 1: 0.5063605602432861, 2: 3.4914353219137624}

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold=StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# XGB

In [ ]:
from xgboost import XGBClassifier
from sklearn import metrics
#xgb = XGBClassifier(random_state=1,max_depth=14,n_estimators=1800,learning_rate=0.45,objective='multi:softmax',class_weight=weights)
xgb = XGBClassifier(random_state=1,max_depth=14,n_estimators=1900,learning_rate=0.3,objective='multi:softmax',class_weight=weights)
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict(X_localtest)
print("f1 score for xgb:",metrics.f1_score(y_localtest,y_pred_xgb,average='weighted'))
#f1 score for xgb: 0.8572959099368004

f1 score for xgb: 0.8551293027825341


In [ ]:
params={
 "learning_rate"    : [0.03,0.05,0.07] ,
 "max_depth"        : [8,10,12],
 "n_estimators"     : [800,1000,1200]
}
classifier=XGBClassifier(random_state=0,objective='multi:softmax',class_weight=weights)
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
grid_search=GridSearchCV(classifier,param_grid=params,scoring='accuracy',n_jobs=-1,cv=kfold,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 120.0min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 274.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     class_weight={0: 1.3537157906790336,
                                                   1: 0.5063605602432861,
                                                   2: 3.4914353219137624},
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weigh...
                                     nthread=None, objective='multi:softmax',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
           

In [ ]:
print(grid_search.best_params_)

{'learning_rate': 0.07, 'max_depth': 8, 'n_estimators': 1200}


In [ ]:
xgb_tuned=grid_search.best_estimator_
xgb_tuned.fit(X_train,y_train)
y_pred_xgb_tuned=xgb_tuned.predict(X_localtest)
print("f1 score for xgb tuned:",metrics.f1_score(y_localtest,y_pred_xgb_tuned,average='weighted'))

f1 score for xgb tuned: 0.8463842437784218


# LGBM

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=2,class_weight=weights,classes_count=3,n_estimators=800,objective="multiclass",boosting_type='dart')
lgb.fit(X_train,y_train)
y_pred_lgb=lgb.predict(X_localtest)
print("Accuracy for lgb:",metrics.accuracy_score(y_localtest, y_pred_lgb))
print("f1 score for lgb:",metrics.f1_score(y_localtest,y_pred_lgb,average='weighted'))

Accuracy for lgb: 0.7628445497006776
f1 score for lgb: 0.7883026951255586


In [ ]:
params={
 "learning_rate"    : [0.05,0.1, 0.2] ,
 "boosting_type"    : ['gbdt','dart'],
 "num_leaves"       : [20,25,31],
 "max_depth"        : [-1, 1, 3],
 "n_estimators"     : [200,300,400]
}
classifier=LGBMClassifier(random_state=2,class_weight=weights,classes_count=3,objective='multiclass')
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
grid_search=GridSearchCV(classifier,param_grid=params,scoring='accuracy',cv=kfold,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=20 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=20, total=   3.5s
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=20 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s


[CV]  boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=20, total=   3.5s
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=20 
[CV]  boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=20, total=   3.5s
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=25 
[CV]  boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=25, total=   3.8s
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=25 
[CV]  boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=25, total=   3.7s
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=25 
[CV]  boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=25, total=   3.7s
[CV] boosting_type=gbdt, learning_rate=0.05, max_depth=-1, n_estimators=200, num_leaves=31 
[CV] 

[Parallel(n_jobs=1)]: Done 486 out of 486 | elapsed: 97.4min finished


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt',
                                      class_weight={0: 1.3537157906790336,
                                                    1: 0.5063605602432861,
                                                    2: 3.4914353219137624},
                                      classes_count=3, colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      mi...
                                      reg_alpha=0.0, reg_lambda=0.0,
                                      silent=True, subsample=1.0,
                                      subsample_for_bin=200000,
                                 

In [ ]:
print(grid_search.best_params_)
lgb_tuned=grid_search.best_estimator_
lgb_tuned.fit(X_train,y_train)
y_pred_lgb_tuned=lgb_tuned.predict(X_localtest)
print("f1 score for lgb tuned:",metrics.f1_score(y_localtest,y_pred_lgb_tuned,average='weighted'))

{'boosting_type': 'gbdt', 'learning_rate': 0.2, 'max_depth': -1, 'n_estimators': 400, 'num_leaves': 31}
f1 score for lgb tuned: 0.8214740193767726


# Cat

In [ ]:
from catboost import CatBoostClassifier
from sklearn import metrics
cat = CatBoostClassifier(random_state=100,logging_level='Silent',class_weights=weights)
cat.fit(X_train,y_train)
y_pred_cat=cat.predict(X_localtest)
print("Accuracy for cat:",metrics.accuracy_score(y_localtest, y_pred_cat))
print("f1 score for cat:",metrics.f1_score(y_localtest,y_pred_cat,average='weighted'))

Accuracy for cat: 0.7880402605091771
f1 score for cat: 0.807705213549664


In [ ]:
predictions=xgb.predict(test)
predictions

array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
submission= pd.DataFrame(predictions, columns = ['prediction'])
submission['prediction']=submission['prediction'].replace([0,1,2],['loss','win','draw'])
submission

,prediction
0,loss
1,win
2,win
3,win
4,draw
...,...
16885,loss
16886,win
16887,win
16888,win


In [ ]:
from google.colab import files
submission.to_csv('submission_xgboost.csv',index=False) 
files.download('submission_xgboost.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>